In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import trange
import re
from lmfit.models import GaussianModel, StepModel, Model
from cn_tools.cns_fit import cns_pc_from_file
from cn_tools.cns_fit import mu_from_gauss
from cn_tools.cns_fit import sigma_from_cns
from cn_tools.cns_fit import Z_from_cns
from cn_tools.cns_fit import compute_local_contacts
from cn_tools.cns_fit import compute_nearest_neighbours
from cn_tools.cns_fit import compute_mean_sigma
from cn_tools.cns_fit import approximate_diameter
from cn_tools.data_processing import cut_df

In [2]:
directories = ['../Data/preprocessed/VF_005_analysis/', '../Data/preprocessed/VF_006_analysis/',
               '../Data/preprocessed/VF_007_analysis/', '../Data/preprocessed/VF_008_analysis/']

In [3]:
def compute_sigmas(directory):
    '''Iterates through a raw directory and saves all processed csv files in another directory called preprocessed'''
    
    #define save directory
    save_file = '../Data/preprocessed/sigma.csv'
    df_sigma = pd.read_csv(save_file, index_col=0)
        
    #create list of all minkowski file folders to evaluate
    files = [directory + file for file in os.listdir(directory) if 'tomo' in file]
    
    #for folder in folders:
    for i in trange(len(files)):
        
        file_name = files[i]
        #index has the form 'XXX_YYY'
        index = file_name[-11:-4]
        interval, cns, pc = cns_pc_from_file(file_name)
        mu = mu_from_gauss(interval, pc)
        sigma = sigma_from_cns(interval, cns, mu)
        df_sigma.loc[index, 'sigma'] = sigma
        
    df_sigma.to_csv(save_file)

In [4]:
def contacts_and_neighbours(directory):
    #final directoryreturn_length
    final_dir = re.sub('preprocessed', 'final', directory)
    if not os.path.isdir(final_dir):
        os.mkdir(final_dir)
    #compute mean sigmas
    mean_sigmas = {res:compute_mean_sigma(resolution=res) for res in [20,30]}
    #pick relevant sigma according to directory name
    sigma = mean_sigmas[approximate_diameter(directory)]
    
    files = [directory + file for file in os.listdir(directory) if 'tomo' in file]
    
    for i in trange(len(files)):
        file_name = files[i]
        poly_file = re.sub('preprocessed', 'raw', file_name)
        poly_file = re.sub('.csv', '.poly', poly_file)
        final_file = re.sub('preprocessed', 'final', file_name)
        interval, cns, pc = cns_pc_from_file(file_name)
        mu = mu_from_gauss(interval, pc)
        Z = Z_from_cns(interval, cns, mu, sigma)
        #print('Z=',Z)
        compute_local_contacts(file_name, Z)
        compute_nearest_neighbours(file_name, poly_file)
        
        df = pd.read_csv(file_name, index_col = 0)
        df['diameter'] = mu
        df.to_csv(final_file)

In [7]:
#directories  = ['../Data/preprocessed/VF_008_analysis_ref_centroid/']
for directory in directories:
    compute_sigmas(directory)

100%|█████████████████████████████████████████| 308/308 [02:43<00:00,  1.88it/s]


In [5]:
for directory in directories:
    contacts_and_neighbours(directory)

100%|█████████████████████████████████████████| 308/308 [15:12<00:00,  2.96s/it]


In [5]:
file_name = '../Data/preprocessed/VF_006_analysis/tomo_gray_SUB_006_003.csv'

df = pd.read_csv(file_name, index_col = 0)
rho = df.shape[0]/df.w000.sum()
xyz = df[['x','y','z']].to_numpy()

#compute approximate diameter (depends on the resolution of the measurement)
d_appr = approximate_diameter(file_name)

#compute data
from scipy.spatial import KDTree
dr = 0.005
interval = np.arange(d_appr-1, d_appr+2, dr)
tree = KDTree(xyz)
%time dists = tree.count_neighbors(tree, interval)
cns_av = dists/xyz.shape[0]-1
pc_av = np.diff(dists, prepend=dists[0])/(4*np.pi*interval**2*dr)
mu = mu_from_gauss(interval, pc)
sigma = sigma_from_cns(interval, cns, mu)
import scipy
scipy.__version__

#!pip install scipy==1.7.0

CPU times: user 536 ms, sys: 9.19 ms, total: 545 ms
Wall time: 547 ms


NameError: name 'pc' is not defined

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2021-07-12 18:39:37.140615: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Num GPUs Available:  0


2021-07-12 18:39:38.511865: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-12 18:39:38.578459: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2021-07-12 18:39:38.578538: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ansgar-HP
2021-07-12 18:39:38.578550: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ansgar-HP
2021-07-12 18:39:38.579008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.42.1
2021-07-12 18:39:38.579065: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.80.0
2021-07-12 18:39:38.579074: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 460.80.0 does not match DSO version 470.42.1 -- can

[]